# Dependências

# 1.Extração de Texto de PDFs

In [27]:
import fitz  # PyMuPDF

def extract_text_pymupdf(pdf_path):
  text = ""
  try:
    with fitz.open(pdf_path) as doc:
      for page in range(len(doc)):
        page_text = doc.load_page(page)
        text += page_text.get_text("text")
  except Exception as e:
    print(f"Error reading {pdf_path} with PyMuPDF: {e}")
    return None
  return text  

# 2.Pré-processamento do Texto Extraído

In [28]:
import spacy
import re

try:
  nlp = spacy.load("pt_core_news_lg")
  nlp.max_length = 3000000
  print("INFO: spaCy model loaded.")
except OSError:
  print("ERROR: 'pt_core_news_lg' not found. Trying 'pt_core_news_md.")
  try:
    nlp = spacy.load("pt_core_news_md")
    print("INFO: spaCy model loaded.")
    print("INFO: 'pt_core_news_md' model is smaller and may not perform as well as 'pt_core_news_lg'.")
  except OSError:
    print("ERROR: 'pt_core_news_md' not found. Please install a spaCy model.")
    print("Run 'pipenv run python3 -m spacy download pt_core_news_lg' to install the model.")
    exit()

INFO: spaCy model loaded.


In [29]:
def clear_text(text):
  if text is None: return ""
  text = re.sub(r'-\n', '', text)
  text = re.sub(r'\[\d+\]', '', text)  # Remove [number] patterns
  text = re.sub(r'\(\d+\)', '', text)  # Remove (number) patterns
  text = re.sub(r'\([\w\s,.]+\d{4}\)', '', text)  # Remove (AUTOR et al., 2020) patterns
  
  lines = text.split('\n')
  cleaned_lines = []
  for line in lines:
    if re.search(r'Rev\. Latino-Am\. Enfermagem|www\.eerp\.usp\.br/rlae|ISSN:|DOI:', line):
      continue
    if re.fullmatch(r'\s*\d+\s*', line) or len(line.strip()) < 10:
        continue
    cleaned_lines.append(line.strip())

  text = " ".join(cleaned_lines)
  text = re.sub(r'\s+', ' ', text)
  text = re.sub(r'[ \t]+', ' ', text)
  return text.strip()


In [30]:
def preprocess_sent_tfidf(sent_doc):
  clear_tokens = [
    token.lemma_.lower()
    for token in sent_doc
      if not token.is_stop 
         and not token.is_punct 
         and not token.is_space
         and len(token.lemma_) > 1
  ]
  return " ".join(clear_tokens)

In [31]:
import pycld2 as cld2
def detect_language(text, default_lang='pt'):
  try:
    isReliable, textBytesFound, details = cld2.detect(text)
    
    return details[0][1]
  except Exception:
    return default_lang

# 3.Cálculo e Extração de TF-IDF

In [32]:
from sklearn.feature_extraction.text import TfidfVectorizer
from collections import Counter
import pandas as pd

In [33]:
def compute_tfidf(sentences):
  if not sentences:
    print("No sentences provided for TF-IDF computation.")
    return None, None, []
  
  tfidf_vectorizer = TfidfVectorizer(
    min_df=2, # Ignorar termos que aparecem em menos de 2 sentenças
    # max_df=0.95, # Ignorar termos que aparecem em mais de 95% das sentenças
    # ngram_range=(1, 1) # Incluir apenas unigrams (palavras únicas)
  )
  
  # Aprende o vocabulário e o IDF, e transforma o corpus em matriz TF-IDF
  tfidf_matrix = tfidf_vectorizer.fit_transform(sentences)
  
  # O vocabulário aprendido (lista de termos em ordem)
  feature_names_tfidf = tfidf_vectorizer.get_feature_names_out()
  
  return tfidf_vectorizer, feature_names_tfidf, tfidf_matrix

In [34]:
# --- Extração da Frequência Bruta Total (Global) ---
def brute_frequency_global(sentences):
  if not sentences:
    print("No sentences provided for global frequency computation.")
    return pd.DataFrame(columns=['Term', 'Frequency'])
  
  print("INFO: Computing global frequency of terms...")
  all_lemmas_corpus = []
  for sent in sentences:
    sent_lemmas = [
      token.lemma_.lower()
      for token in sent
        if not token.is_stop 
           and not token.is_punct 
           and not token.is_space
           and len(token.lemma_) > 1
    ]
    all_lemmas_corpus.extend(sent_lemmas)
  
  total_lemmas_count = Counter(all_lemmas_corpus)
  terms_ordered_by_frequency_brute = total_lemmas_count.most_common()
  df_brute_frequency = pd.DataFrame(terms_ordered_by_frequency_brute, columns=['Term', 'Frequency'])
  
  print(f"INFO: Brute frequency computed for {len(total_lemmas_count)} terms.")
  print("INFO: Top 20 terms by frequency:", terms_ordered_by_frequency_brute[:20])
  return df_brute_frequency

In [35]:
# --- Extração da Importância Total TF-IDF (Global) ---
def tfidf_importance_global(feature_names_tfidf, tfidf_matrix):
  if tfidf_matrix is None and feature_names_tfidf is None and tfidf_matrix.shape[0] == 0:
    print("No valid TF-IDF data provided for global importance computation.")
    return pd.DataFrame(columns=['Term', 'Importance'])

  print("INFO: Computing global TF-IDF importance of terms...")
  
  tfidf_sum = tfidf_matrix.sum(axis=0).tolist()[0]
  terms_importance = {}
  for i, term in enumerate(feature_names_tfidf):
    terms_importance[term] = tfidf_sum[i]
  
  terms_importance_ordered = sorted(terms_importance.items(), key=lambda item: item[1], reverse=True)
  df_tfidf_importance = pd.DataFrame(terms_importance_ordered, columns=['Term', 'Importance'])
  
  print(f"INFO: Global TF-IDF importance computed for {len(terms_importance)} terms.")
  print("INFO: Top 20 terms by TF-IDF importance:", terms_importance_ordered[:20])
  return df_tfidf_importance

In [36]:
# --- Preparar Dados para Planilha de Termos ---
# Criar um DataFrame para consolidar as informações
def prepare_terms_dataframe(terms_brute_df, terms_importance_df):
  if terms_brute_df.empty or terms_importance_df.empty:
    print("No terms provided for DataFrame preparation.")
    return pd.DataFrame()
  
  print("INFO: Preparing DataFrame for terms...")
  
  df_terms = pd.merge(terms_brute_df, terms_importance_df, on='Term', how='outer').fillna(0)
  # Remover termos com frequência ou importancia zero
  df_terms = df_terms[df_terms['Frequency'] > 0]
  df_terms = df_terms[df_terms['Importance'] > 0]
  
  # Remover termos com apenas números
  df_terms = df_terms[~df_terms['Term'].str.match(r'^\d+$')]

  return df_terms
  

# 4.Extração de Relações de Causa e Efeito

In [37]:
# --- Dicionários de Verbos Causais ---
# TODO: substituir por leitura de arquivo externo
CAUSAL_VERBS_LEMAS = {
  "causar", "provocar", "ocasionar", "gerar", "levar", "resultar",
  "desencadear", "induzir", "promover", "acarretar", "implicar", 
  "produzir", "motivar", "suscitar", "originar", "contribuir",
  "aumentar", "reduzir", "elevar", "diminuir"
}

CAUSAL_NOUNS_LEMAS = {
  "fator", "motivo", "razão", "causa", "preditor", "marcador", "risco", 
}

EFFECT_NOUNS_LEMAS = {
  "consequencia", "resultado", "efeito", "impacto", "repercussão", "desfecho"
}

In [38]:
def clear_action_words(text):
  doc = nlp(text)
  filtered_tokens = []
  ACTION_LEMMAS = { "aumentar", "reduzir", "elevar", "elevação", "diminuir", "queda"
                   "crescer", "crescimento", "diminuição", "redução" }
  for token in doc:
    if token.lemma_.lower() not in ACTION_LEMMAS:
      filtered_tokens.append(token.text_with_ws)
  return "".join(filtered_tokens).strip()

In [39]:
def is_valid_entity(entity, nlp_doc):
  if len(entity) < 3:
    return False
  if re.search(r'\d{4}|[\[\(]\d+[\]\)]|http|www|ISSN|DOI', entity):
    return False
  
  doc = nlp(entity)
  
  non_stop_tokens = [token for token in doc if not token.is_stop and not token.is_punct and not token.is_space]
  if not non_stop_tokens:
    return False
  
  has_content_word = any(token.pos_ in ['NOUN', 'PROPN', 'ADJ'] for token in doc)
  if not has_content_word:
    return False
  
  return True

In [40]:
def extract_causal_relations(sent_doc):
  if sent_doc is None: return []
  
  relations = []
  
  def get_entity_phrase(token):
    tokens_in_phrase = []
    for t in token.subtree:
      if t.dep_ in ('relcl', 'advcl') and t != token:
        break
      tokens_in_phrase.append(t.text)
    return " ".join(tokens_in_phrase)
  
  for token in sent_doc:
    # --- Lógica 1: Verbos Causais (Causa -> Verbo -> Efeito) 
    if token.lemma_ in CAUSAL_VERBS_LEMAS and token.pos_ == "VERB":
      subjects = [child for child in token.children if child.dep_ in ("nsubj", "nsubj:pass")]
      objects = [child for child in token.children if child.dep_ in ("obj", "dobj")]
      obl_objects = [grandchild for child in token.children if child.dep_ == "obl" for grandchild in child.children if grandchild.dep_ == "pobj"]
      objects.extend(obl_objects)
      
      for subj in subjects:
        if len(list(subj.subtree)) > 1 or subj.pos_ not in ['PRON', 'DET']:
          for obj in objects:
            cause = get_entity_phrase(subj)
            effect = get_entity_phrase(obj)
            
            if cause and effect:
              cause = clear_action_words(cause)
              effect = clear_action_words(effect)
              if is_valid_entity(cause, sent_doc) and is_valid_entity(effect, sent_doc):
                relations.append({
                  "tipo_marcador": "verbo", "marcador": token.text,
                  "lema_marcador": token.lemma_, "causa": cause, "efeito": effect,
                  "direcao": "Causa -> Efeito", "sentenca_original": sent_doc.text
                })
    
    # --- Lógica 2: Substantivos Causais (Causa -> Subst -> Efeito) ---
    elif token.pos_ == "NOUN" and (token.lemma_ in CAUSAL_NOUNS_LEMAS or token.lemma_ in EFFECT_NOUNS_LEMAS):
      is_cause_noun = token.lemma_ in CAUSAL_NOUNS_LEMAS
      
      main_entity_token = None
      secondary_entity_token = None
      
      for child in token.children:
        if child.dep_ == 'nmod':
          secondary_entity_token = child
          break
      
      if token.dep_ == 'attr':
        main_entity_token = next((child for child in token.head.children if child.dep_ == 'nsubj'), None)
      elif token.dep_ == 'nsubj':
        main_entity_token = next((child for child in token.head.children if child.dep_ == 'attr'), None)
      elif token.dep_ == 'ROOT':
        main_entity_token = next((child for child in token.children if child.dep_ == 'appos'), None)
        
      if main_entity_token and secondary_entity_token:
        main_entity_text = get_entity_phrase(main_entity_token)
        secondary_entity_token = get_entity_phrase(secondary_entity_token)
        
        if is_cause_noun:
          cause, effect = main_entity_text, secondary_entity_token
          direction = "Causa -> Efeito"
        else:
          cause, effect = secondary_entity_token, main_entity_text
          direction = "Efeito <- Causa"
          
        if cause and effect:
          cause = clear_action_words(cause)
          effect = clear_action_words(effect)
          if is_valid_entity(cause, sent_doc) and is_valid_entity(effect, sent_doc):
            relations.append({
              "tipo_marcador": "substantivo (" + ("causa" if is_cause_noun else "efeito") + ")",
              "marcador": token.text, "lema_marcador": token.lemma_,
              "causa": cause, "efeito": effect,
              "direcao": direction, "sentenca_original": sent_doc.text
            })
  
  unique_relations = []
  seen = set()
  for rel in relations:
    identifier = (rel['causa'], rel['efeito'], rel['direcao'])
    if identifier not in seen:
      unique_relations.append(rel)
      seen.add(identifier)
  
  return unique_relations

# 5. Enriquecer as Relações com TF-IDF

In [41]:
def enrich_relations_with_tfidf(relations, tfidf_vectorizer):
  if tfidf_vectorizer is None:
    print("TF-IDF vectorizer is not provided. Skipping enrichment.")
    return relations
  
  vocabulary = {term: tfidf_vectorizer.idf_[idx] for term, idx in tfidf_vectorizer.vocabulary_.items()}
  
  enriched_relations = []
  
  for rel in relations:
    new_relation = rel.copy()
    
    cause_doc = nlp(rel['causa'])
    effect_doc = nlp(rel['efeito'])
    
    cause_terms = [t.lemma_.lower() for t in cause_doc if not t.is_stop and not t.is_punct]
    effect_terms = [t.lemma_.lower() for t in effect_doc if not t.is_stop and not t.is_punct]
    
    score_cause_idf = []
    score_effect_idf = []

    for term in cause_terms:
      if term in vocabulary:
        score_cause_idf.append(vocabulary[term])
    for term in effect_terms:
      if term in vocabulary:
        score_effect_idf.append(vocabulary[term])

    score_cause = sum(score_cause_idf) / len(score_cause_idf) if score_cause_idf else 0
    score_effect = sum(score_effect_idf) / len(score_effect_idf) if score_effect_idf else 0
    
    total_score = score_cause + score_effect
    
    new_relation['score_relacao'] = round(total_score, 2)
    new_relation['termos_causa'] = ", ".join(cause_terms)
    new_relation['termos_efeito'] = ", ".join(effect_terms)
    enriched_relations.append(new_relation)
  
  enriched_relations.sort(key=lambda x: x['score_relacao'], reverse=True)
  return enriched_relations

# Workflow

## 1. Extração de Texto de PDFs

In [42]:
from pathlib import Path
# 1. Extrair texto bruto do PDF
CORPUS_DIR = Path("corpus")
RECURSIVE = True

TERMS_OUTPUT = "terms_analysis.csv"
RELATIONS_OUTPUT = "cause_effect_relations.csv"

brute_texts = []
processed_files = []

if RECURSIVE:
  search_pattern = CORPUS_DIR.rglob("*.pdf")
else:
  search_pattern = CORPUS_DIR.glob("*.pdf")

for pdf_path in search_pattern:
  print(f"Processando arquivo: {pdf_path}")
  extracted_text = extract_text_pymupdf(pdf_path)
  
  if extracted_text:
    brute_texts.append(extracted_text)
    processed_files.append(pdf_path.name)
    print(f"  ... Texto extradio com sucesso ({len(extracted_text)} caracteres)")
  else:
    print(f"Falha ao extrair texto de: {pdf_path}")
    continue

print(f"Processamento concluido. {len(brute_texts)} arquivos processados.")

if not brute_texts:
  print("Nenhum texto foi extraído dos PDFs.")
  exit()

Processando arquivo: corpus\ncomms5212.pdf
  ... Texto extradio com sucesso (59025 caracteres)
Processamento concluido. 1 arquivos processados.


## 2.Pré-processamento do Texto Extraído

In [43]:
# 2. Limpeza Inicial do Texto Bruto
print("\nIniciando pré-processamento de todo o corpus...")
sentences_docs = []
info_sentences = []
total_ignored = 0

for i, brute_text in enumerate(brute_texts):
  text_name = processed_files[i]
  cleared_text = clear_text(brute_text)
  doc_text = nlp(cleared_text)
  for sent in doc_text.sents:
    detected_lang = detect_language(sent.text)
    if detected_lang != 'pt':
      total_ignored += 1
      continue
    sentences_docs.append(sent)
    info_sentences.append({"arquivo_origem": text_name})

print(f"Pré-processamento concluído. {len(sentences_docs)} sentenças extraídas do corpus.")
print(f"Total de sentenças ignoradas (não em português): {total_ignored}")



Iniciando pré-processamento de todo o corpus...
Pré-processamento concluído. 9 sentenças extraídas do corpus.
Total de sentenças ignoradas (não em português): 1006


In [44]:
# 3. Preparar lista de sentenças limpas para TF-IDF
sentences_tfidf = []
for sent_doc in sentences_docs:
    sentences_tfidf.append(preprocess_sent_tfidf(sent_doc))

### **VISUALIZAÇÃO DAS SENTENÇAS**

In [45]:
for i, sent_doc in enumerate(sentences_docs):
    print(f"Sentença {i+1}: {sent_doc}")

Sentença 1: The Pearson correlation coefﬁcient (PCC) between the ratio of shared disease links and disease similarity is very high (PCC ¼ 0.96, P ¼ 1.4  10  5), indicating that the proposed disease similarity is a reliable measure for shared symptoms.
Sentença 2: The number of overlapping links is signiﬁcantly (P ¼ 2.2  10  16, Supplementary Methods) than random expectation, again indicating that the HSDN offers reliable relationships.
Sentença 3: 8-fold increase compared with random expectation, P ¼ 2.2  10  16, binomial test;
Sentença 4: shows strong positive correlation with disease similarity (PCC ¼ 0.92 and P ¼ 1.8  10  4;
Sentença 5: The ratio of diseases with shared PPIs increases signiﬁcantly with higher symptom similarity (PCC ¼ 0.89, P ¼ 5.4  10  4 for 1st order interactions, Fig.
Sentença 6: Indeed, we ﬁnd strong negative correlation between the MSPL and symptom similarities (PCC ¼  0.93 and P ¼ 7.7  10  5;
Sentença 7: is statistically highly signiﬁcant (P ¼ 2.2

In [46]:
print("\n--- Sentenças Limpas para TF-IDF ---")
for i, sent_limpa in enumerate(sentences_tfidf):
    print(f"Sentença {i+1} (limpa): {sent_limpa}")


--- Sentenças Limpas para TF-IDF ---
Sentença 1 (limpa): the pearson correlation coefﬁcient pcc between the ratio of shared disease link and disease similarity is very high pcc 0.96 1.4 10 indicating that the proposed disease similarity is reliable measure shared symptoms
Sentença 2 (limpa): the number of overlapping link is signiﬁcantly 2.2 10 16 supplementary methods than random expectation again indicating that the hsdn offers reliable relationships
Sentença 3 (limpa): 8-fold increase compared with random expectation 2.2 10 16 binomial test
Sentença 4 (limpa): show strong positive correlation with disease similarity pcc 0.92 and 1.8 10
Sentença 5 (limpa): the ratio of diseases with shared ppis increases signiﬁcantly with higher symptom similarity pcc 0.89 5.4 10 1st order interactions fig
Sentença 6 (limpa): indeed we ﬁnd strong negative correlation between the mspl and symptom similarities pcc 0.93 and 7.7 10
Sentença 7 (limpa): is statistically highly signiﬁcant 2.2 10 16 binomia

## 3.Cálculo e Extração de TF-IDF

In [47]:
# 4. Calcular TF-IDF
tfidf_vectorizer, feature_names_tfidf, tfidf_matrix = compute_tfidf(sentences_tfidf)
if tfidf_vectorizer is not None:
    print(f"TF-IDF calculado. Corpus com {tfidf_matrix.shape[0]} sentenças e {tfidf_matrix.shape[1]} termos únicos.")
    print("\nScores TF-IDF para a primeira sentença:")
    primeira_sentenca_tfidf_scores = tfidf_matrix[0].T.todense() # Converte para matriz densa
    termos_scores = [(feature_names_tfidf[i], primeira_sentenca_tfidf_scores[i, 0]) for i in range(len(feature_names_tfidf)) if primeira_sentenca_tfidf_scores[i, 0] > 0]
    print(sorted(termos_scores, key=lambda item: item[1], reverse=True)[:10]) # Top 10 scores na primeira sentença
else:
    print("Erro ao computar TF-IDF. Verifique as sentenças fornecidas.")

TF-IDF calculado. Corpus com 9 sentenças e 27 termos únicos.

Scores TF-IDF para a primeira sentença:
[('disease', np.float64(0.5140215757590505)), ('the', np.float64(0.39488426537531346)), ('shared', np.float64(0.34268105050603365)), ('is', np.float64(0.2979512813337321)), ('similarity', np.float64(0.2979512813337321)), ('pcc', np.float64(0.21094035011304343)), ('between', np.float64(0.17134052525301682)), ('indicating', np.float64(0.17134052525301682)), ('link', np.float64(0.17134052525301682)), ('ratio', np.float64(0.17134052525301682))]


In [48]:
# 5. Extração da Frequência Bruta Total (Global)
terms_brute_df = brute_frequency_global(sentences_docs)
# 6. Extração da Importância Total TF-IDF (Global)
terms_importance_df = tfidf_importance_global(feature_names_tfidf, tfidf_matrix)
# 7. Criar DataFrame com os resultados
df_terms = prepare_terms_dataframe(terms_brute_df, terms_importance_df)

df_terms.to_csv('terms_analysis.csv', index=False, encoding='utf-8-sig')
print("INFO: DataFrame prepared and saved to 'terms_analysis.csv'.")

INFO: Computing global frequency of terms...
INFO: Brute frequency computed for 80 terms.
INFO: Top 20 terms by frequency: [('10', 10), ('the', 7), ('pcc', 7), ('correlation', 5), ('disease', 4), ('and', 4), ('similarity', 4), ('is', 4), ('with', 4), ('of', 3), ('shared', 3), ('2.2', 3), ('16', 3), ('strong', 3), ('fig', 3), ('between', 2), ('ratio', 2), ('link', 2), ('indicating', 2), ('that', 2)]
INFO: Computing global TF-IDF importance of terms...
INFO: Global TF-IDF importance computed for 27 terms.
INFO: Top 20 terms by TF-IDF importance: [('10', 1.9063008143594562), ('pcc', 1.6351667229209863), ('correlation', 1.4916212618258036), ('the', 1.360898862898396), ('fig', 1.3399586075048224), ('with', 1.2537699746614224), ('and', 1.1398140531508894), ('strong', 1.0968038874933725), ('16', 1.0672884470561192), ('is', 1.0016265256868295), ('binomial', 0.936827742256019), ('test', 0.936827742256019), ('disease', 0.919621430575187), ('similarity', 0.9193583500235566), ('positive', 0.894467

In [49]:
# Terms ordered by frequency
print("\n--- Top 20 Terms by Importance ---")
print(df_terms.sort_values(by='Importance', ascending=False).head(10))


--- Top 20 Terms by Importance ---
           Term  Frequency  Importance
51          pcc          7    1.635167
25  correlation          5    1.491621
74          the          7    1.360899
30          fig          3    1.339959
77         with          4    1.253770
19          and          4    1.139814
67       strong          3    1.096804
40           is          4    1.001627
22     binomial          2    0.936828
71         test          2    0.936828


## 4.Extração de Relações de Causa e Efeito

In [50]:
# 8. Extração de Relações Causais
all_relations_found = []
for i, sent_doc in enumerate(sentences_docs):
  sentence_relations = extract_causal_relations(sentences_docs[i])
  if sentence_relations:
    for rel in sentence_relations:
      rel['arquivo_origem'] = info_sentences[i]['arquivo_origem']
    all_relations_found.extend(sentence_relations)
    print(f"Encontradas {len(sentence_relations)} relações causais na sentença {i+1}: \"{sent_doc.text}\".")

print(f"Total de sentenças processadas: {len(sentences_docs)}")
# 9. Exibir as relações causais encontradas
print("\n--- Resumo das Relações Causais Encontradas ---")
if all_relations_found:
  df_relations = pd.DataFrame(all_relations_found)
  if not df_relations.empty:
    ordered_columns = ['tipo_marcador', 'lema_marcador', 'causa', 'marcador', 'efeito', 'direcao', 'sentenca_original', 'arquivo_origem']
    df_relations = df_relations[ordered_columns]
    df_relations.to_csv("cause_effect_relations.csv", index=False, encoding='utf-8-sig')
    print("Arquivo 'cause_effect_relations.csv' gerado.")

    print("\nPrimeiras 10 relações causais encontradas:")
    print(df_relations.head(10))
  else:
    print("Nenhuma relação de causa e efeito única encontrada para gerar o CSV.")
else:
  print("Nenhuma relação de causa e efeito encontrada no corpus.")

Total de sentenças processadas: 9

--- Resumo das Relações Causais Encontradas ---
Nenhuma relação de causa e efeito encontrada no corpus.


## 5. Enriquecer as Relações com TF-IDF

In [51]:
print("\n--- Enriquecendo Relações com Scores TF-IDF ---")

enriched_relations = enrich_relations_with_tfidf(all_relations_found, tfidf_vectorizer)

if enriched_relations:
    df_enriched_relations = pd.DataFrame(enriched_relations)
    if not df_enriched_relations.empty:
        ordered_columns = ['score_relacao', 'direcao', 'tipo_marcador',
            'causa', 'marcador', 'efeito', 'termos_causa', 'termos_efeito', 'sentenca_original','arquivo_origem']
        final_columns = [col for col in ordered_columns if col in df_enriched_relations.columns]
        df_enriched_relations = df_enriched_relations[final_columns]
        
        df_enriched_relations.to_csv("enriched_cause_effect_relations.csv", index=False, encoding='utf-8-sig')
        print("Arquivo 'enriched_cause_effect_relations.csv' gerado com sucesso.")
        print("\nPrimeiras 10 relações enriquecidas:")
        print(df_enriched_relations[['score_relacao', 'causa', 'efeito', 'direcao']].head(10))
    else:
        print("Nenhuma relação enriquecida encontrada para gerar o CSV.")
else:
    print("Nenhuma relação causal encontrada para enriquecer com TF-IDF.")
        
    


--- Enriquecendo Relações com Scores TF-IDF ---
Nenhuma relação causal encontrada para enriquecer com TF-IDF.
